In [89]:
import numpy as np
import os as os
import torch
import torch.nn as nn
import cv2
from PIL import Image

In [132]:
# Define the model architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x
    
model = CNN()

input_tensor = torch.randn(1, 3, 70, 70)

output = model.forward(input_tensor)
print(np.array(input_tensor).shape)

(1, 3, 70, 70)


In [108]:
def create_images(img_folder, new_folder):
    
    for dir1 in os.listdir(img_folder):
        
        if dir1 != '.DS_Store':
            
            for file in os.listdir(os.path.join(img_folder, dir1)):
                
                new_name = os.path.splitext(file)
                
                
                image_path = os.path.join(img_folder, dir1)
                new_image_path = os.path.join(new_folder, dir1)

                im1 = Image.open('{}/{}'.format(image_path, file))

                rbg_im1 = im1.convert('RGB')


                rbg_im1.save('{}/{}.jpg'.format(new_image_path, new_name[0]))
                
    return 0

create_images('t_or_l_images', 'fixed_images')

0

In [152]:
def create_dataset(img_folder):
   
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        if dir1 != '.DS_Store':
            for file in os.listdir(os.path.join(img_folder, dir1)):
                
                
       
                image_path= os.path.join(img_folder, dir1,  file)
                image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
                image=cv2.resize(image, (70, 70),interpolation = cv2.INTER_AREA)
                image=np.array(image)
                image = image.astype('float32')
                image /= 255 
                img_data_array.append(image)
                class_name.append(dir1)
                
    return img_data_array, class_name# extract the image array and class name

dataset_X, dataset_y = create_dataset('fixed_images')


In [153]:
dataset_X = np.array(dataset_X)

In [154]:


shape = dataset_X.shape

print(dataset_X.shape)

dataset_X = np.reshape(dataset_X, (shape[0],shape[3],shape[1],shape[2]))

print(dataset_X.shape)

output = model.forward(torch.tensor(dataset_X))

print()

(573, 70, 70, 3)
(573, 3, 70, 70)


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
# Compile the model

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# DO NOT RUN THIS IF YOU WANT TO KEEP KERNEL ALIVE

import tensorflow as tf
from tensorflow.keras import layers

# Define the model architecture
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])